In [56]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import BatchNorm1d


class GRUModel(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, num_layers=3, dropout=0.5):
        super(GRUModel, self).__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=0)
        
        # Single-directional GRU
        self.gru = nn.GRU(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
        )

        # Batch normalization
        self.batchnorm = BatchNorm1d(hidden_dim)

        # Fully connected layers
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Embedding layer
        embedded = self.embedding(x)

        # GRU layer
        gru_out, _ = self.gru(embedded)

        # Batch normalization
        gru_out = gru_out.permute(0, 2, 1)  # Reshape for BatchNorm1d
        gru_out = self.batchnorm(gru_out)
        gru_out = gru_out.permute(0, 2, 1)  # Reshape back to original

        # Fully connected layer
        output = self.fc2(gru_out)

        return F.log_softmax(output, dim=-1)



In [2]:
import pandas as pd
train_path = '../dataset/PIZZA_train.json'
test_path = '../dataset/PIZZA_dev.json'
df = pd.read_json(train_path, lines=True)
dev = pd.read_json(test_path, lines=True)

In [3]:
df.describe()

,train.SRC,train.EXR,train.TOP,train.TOP-DECOUPLED
count,2456446,2456446,2456446,2456446
unique,2456446,694346,2456446,1425035
top,can i have a large bbq pulled pork,(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE PARTY_SIZ...,(ORDER can i have (PIZZAORDER (NUMBER a ) (SIZ...,(ORDER (PIZZAORDER (NUMBER three ) (NOT (TOPPI...
freq,1,1999,1,167


In [4]:
unique_exr = df['train.EXR'].value_counts()[df['train.EXR'].value_counts() == 1].index

# Filter rows where 'train.EXR' is unique
unique_related_dataset = df[df['train.EXR'].isin(unique_exr)].reset_index(drop=True)
unique_related_dataset

,train.SRC,train.EXR,train.TOP,train.TOP-DECOUPLED
0,party sized high rise dough pie with american ...,(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE PARTY_SIZ...,(ORDER (PIZZAORDER (SIZE party sized ) (STYLE ...,(ORDER (PIZZAORDER (SIZE party sized ) (STYLE ...
1,meatlover pie with extra chicken,(ORDER (PIZZAORDER (NUMBER 1 ) (STYLE MEAT_LOV...,(ORDER (PIZZAORDER (STYLE meatlover ) pie with...,(ORDER (PIZZAORDER (STYLE meatlover ) (COMPLEX...
2,medium high rise dough pie with artichoke,(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE MEDIUM ) ...,(ORDER (PIZZAORDER (SIZE medium ) (STYLE high ...,(ORDER (PIZZAORDER (SIZE medium ) (STYLE high ...
3,large pie with green pepper and peperonni and ...,(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE LARGE ) (...,(ORDER (PIZZAORDER (SIZE large ) pie with (TOP...,(ORDER (PIZZAORDER (SIZE large ) (TOPPING gree...
4,large pie with chicken and mozzarella and ranc...,(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE LARGE ) (...,(ORDER (PIZZAORDER (SIZE large ) pie with (TOP...,(ORDER (PIZZAORDER (SIZE large ) (TOPPING chic...
...,...,...,...,...
476365,i'd like a pizza with pesto mushrooms and gree...,(ORDER (PIZZAORDER (NUMBER 1 ) (TOPPING PESTO ...,(ORDER i'd like (PIZZAORDER (NUMBER a ) pizza ...,(ORDER (PIZZAORDER (NUMBER a ) (TOPPING pesto ...
476366,i'd like a pizza with arugula ricotta cheese a...,(ORDER (PIZZAORDER (NUMBER 1 ) (TOPPING ARUGUL...,(ORDER i'd like (PIZZAORDER (NUMBER a ) pizza ...,(ORDER (PIZZAORDER (NUMBER a ) (TOPPING arugul...
476367,i'd like a pizza with yellow peppers fried oni...,(ORDER (PIZZAORDER (NUMBER 1 ) (TOPPING YELLOW...,(ORDER i'd like (PIZZAORDER (NUMBER a ) pizza ...,(ORDER (PIZZAORDER (NUMBER a ) (TOPPING yellow...
476368,i'd like a pizza with olives roasted tomatoes ...,(ORDER (PIZZAORDER (NUMBER 1 ) (TOPPING OLIVES...,(ORDER i'd like (PIZZAORDER (NUMBER a ) pizza ...,(ORDER (PIZZAORDER (NUMBER a ) (TOPPING olives...


In [5]:
#unique_patterns = df['train.EXR'].value_counts()
# # Subset the DataFrame for rows with unique values
# unique_related_dataset = df[df['train.EXR'].isin(unique_patterns.index[:5000])] # reduce to 5000 patterns

# # Describe the resulting dataset
# unique_related_dataset.describe()

In [6]:
# unique_related_dataset =unique_related_dataset.reset_index(drop=True)
# unique_related_dataset

In [7]:
X_train = unique_related_dataset['train.SRC']
y_train = unique_related_dataset['train.TOP']
X_test = dev['dev.SRC']
y_test = dev['dev.TOP']
print(len(unique_related_dataset))
print(X_train[476368])
print(y_train[476368])
print(dev['dev.SRC'][0])

476370
i'd like a pizza with olives roasted tomatoes and broccoli without thin crust
(ORDER i'd like (PIZZAORDER (NUMBER a ) pizza with (TOPPING olives ) (TOPPING roasted tomatoes ) and (TOPPING broccoli ) without (NOT (STYLE thin crust ) ) ) )
i want to order two medium pizzas with sausage and black olives and two medium pizzas with pepperoni and extra cheese and three large pizzas with pepperoni and sausage


In [8]:
import sys
sys.path.append("..")
from utils.data_preprocessing import preprocess_text
from utils.feature_extraction import bag_of_words, tfidf_features, extract_embeddings
X_train = [" ".join(preprocess_text(text)) for text in X_train]
X_test = [" ".join(preprocess_text(text)) for text in X_test]
y_train = [" ".join(preprocess_text(text)) for text in y_train]
y_test = [" ".join(preprocess_text(text)) for text in y_test]

In [9]:
X_train

['party sized high rise dough pie with american cheese with extra oregano',
 'meatlover pie with extra chicken',
 'medium high rise dough pie with artichoke',
 'large pie with green pepper pepperoni with extra roasted pepper',
 'large pie with chicken mozzarella ranch sauce',
 'large pie with banana pepper meatball with italian sausage',
 'high rise dough pie with cheese',
 'regular big meat pie with green pepper',
 'party sized pie with american cheese mozzarella with extra tomato sauce',
 'party sized pie with a little bit american cheese with pickle',
 'party sized stuffed crust pie with extra banana pepper with extra meatball',
 'lunch sized pie with a little bit american cheese',
 'regular big meat pie with extra banana pepper pecorino cheese',
 'big new yorker pie with american cheese with olive oil',
 'medium pie with banana pepper pepperoni with a little bit roasted green pepper',
 'lunch sized stuffed crust pie with banana pepper pecorino cheese',
 'personal pie with bbq sauce

In [10]:
y_train[0]

'ORDER PIZZAORDER SIZE party sized STYLE high rise dough pie with TOPPING american cheese with COMPLEX_TOPPING QUANTITY extra TOPPING oregano'

SRC
party sized high rise dough pie with american cheese with oregano
TOP
party sized high rise dough pie american cheese with oregano


ORDER PIZZAORDER SIZE party sized STYLE high rise dough pie with TOPPING american cheese with COMPLEX_TOPPING QUANTITY TOPPING oregano
(ORDER (PIZZAORDER (SIZE party sized ) (STYLE high rise dough ) pie with (TOPPING american cheese ) and with (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING oregano ) ) ) )

i need to order one large pizza with ham bacon onions and black olives one medium pizza with sausage and onions and six large cokes

i need to order one large pizza with ham bacon onions and  black olives one medium pizza with sausage  and  onions and six large cokes




In [11]:
max_str_1 = len(max(X_train, key=len))
max_str_2 = len(max(y_train, key=len))
max_str_1, max_str_2

(116, 267)

In [12]:
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
vocab = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2}  # Special tokens

def tokenize_output(output):
    """
    Tokenizes the structured output into meaningful tokens.
    Example:
        Input: "(ORDER (PIZZAORDER (NUMBER a ) (SIZE large ) (TOPPING bbq pulled pork ) ) )"
        Output: ["(ORDER", "(PIZZAORDER", "(NUMBER", "a", "(SIZE", "large", "(TOPPING", "bbq", "pulled", "pork", ")", ")", ")", ")"]
    """
    tokens = re.findall(r"\(|\)|\w+|[^\s()]+", output)
    return tokens

def build_vocab(outputs, index):
    """
    Builds a vocabulary from tokenized outputs.
    """
    i = index
    for output in outputs:
        tokens = tokenize_output(output)
        for token in tokens:
            if token not in vocab:
                vocab[token] = i
                i += 1
    return vocab, i
def encode_outputs(outputs, vocab):
    """
    Encodes tokenized outputs into sequences of integers.
    """
    encoded = []
    for output in outputs:
        tokens = tokenize_output(output)
        sequence =  [vocab[token] for token in tokens if token in vocab]#[vocab["<SOS>"]] + + [vocab["<EOS>"]]
        encoded.append(sequence)
    return encoded

def pad_sequences_to_fixed_length(sequences, max_len):
    """
    Pads sequences to a fixed length.
    """
    return pad_sequences(sequences, maxlen=max_len, padding="post", value=0)

def decode_sequence(sequence, vocab):
    """
    Decodes a sequence of integers back into the structured output string.
    """
    inv_vocab = {v: k for k, v in vocab.items()}  # Reverse the vocabulary
    tokens = [inv_vocab[idx] for idx in sequence if idx in inv_vocab and idx not in { vocab["<SOS>"], vocab["<EOS>"],vocab["<PAD>"]} ] 
    output = " ".join(tokens)
    output = output.replace(" ( ", " (").replace("( ", "(") #.replace(" )", ")")
    return output

def decode_sequence_2(sequence, vocab):
    """
    Decodes a sequence of integers back into the structured output string.
    """
    inv_vocab = {v: k for k, v in vocab.items()}  # Reverse the vocabulary
    # sequence = sequence.cpu().tolist()  # Convert tensor to a list of integers
    tokens = [inv_vocab.get(idx, "") for idx in sequence if idx > 0]  # Ignore unknown and put empty char
    return "".join(tokens)



In [13]:
# import gensim.downloader as api
# glove_vectors = api.load("glove-wiki-gigaword-100")  # 100-dimension GloVe
# glove_vectors

In [14]:


def prepare_data(
    X_train, y_train, X_test, y_test, feature_type="bow", glove_vectors=None, max_len_1=20, max_len_2 = 20
):

    vectorizer = None

    # # Feature Extraction for X_train and X_test
    # if feature_type == "bow":
    #     X_train_processed, vectorizer = bag_of_words(X_train)
    #     X_train_processed = X_train_processed.toarray()
    #     X_test_processed = vectorizer.transform(X_test).toarray()
    # elif feature_type == "tfidf":
    #     X_train_processed, vectorizer = tfidf_features(X_train)
    #     X_train_processed = X_train_processed.toarray()
    #     X_test_processed = vectorizer.transform(X_test).toarray()
    # elif feature_type == "embeddings":
    #     if not glove_vectors:
    #         raise ValueError("GloVe vectors must be provided for embeddings.")
    #     X_train_tokenized = [sentence.split() for sentence in X_train]
    #     X_test_tokenized = [sentence.split() for sentence in X_test]
    #     X_train_processed = extract_embeddings(X_train_tokenized)
    #     X_test_processed = extract_embeddings(X_test_tokenized)
    # else:
    #     raise ValueError("Invalid feature type. Choose 'bow', 'tfidf', or 'embeddings'.")
    index = 3
    X_vocab, index = build_vocab(X_train,index)  # Build vocabulary from training outputs
    X_train_encoded = encode_outputs(X_train, X_vocab)  # Encode training outputs
    X_test_encoded = encode_outputs(X_test, X_vocab)  # Encode testing outputs
    X_train_processed = pad_sequences_to_fixed_length(X_train_encoded, max_len_1)
    X_test_processed = pad_sequences_to_fixed_length(X_test_encoded, max_len_1)

    y_vocab, index = build_vocab(y_train,index)  # Build vocabulary from training outputs
    y_train_encoded = encode_outputs(y_train, vocab)  # Encode training outputs
    y_test_encoded = encode_outputs(y_test, vocab)  # Encode testing outputs
    y_train_processed = pad_sequences_to_fixed_length(y_train_encoded, max_len_2)
    y_test_processed = pad_sequences_to_fixed_length(y_test_encoded, max_len_2)


    return (
        X_train_processed,
        X_test_processed,
        y_train_processed,
        y_test_processed,
        X_vocab,
        y_vocab,  # Return vocabulary for decoding
    )


In [15]:
X_train_processed, X_test_processed, y_train_processed, y_test_processed, X_vocab, y_vocab = prepare_data( X_train, y_train, X_test, y_test, feature_type="embeddings", max_len_1=270, max_len_2=270)

In [16]:
X_vocab

{'<PAD>': 0,
 '<SOS>': 1,
 '<EOS>': 2,
 'party': 3,
 'sized': 4,
 'high': 5,
 'rise': 6,
 'dough': 7,
 'pie': 8,
 'with': 9,
 'american': 10,
 'cheese': 11,
 'extra': 12,
 'oregano': 13,
 'meatlover': 14,
 'chicken': 15,
 'medium': 16,
 'artichoke': 17,
 'large': 18,
 'green': 19,
 'pepper': 20,
 'pepperoni': 21,
 'roasted': 22,
 'mozzarella': 23,
 'ranch': 24,
 'sauce': 25,
 'banana': 26,
 'meatball': 27,
 'italian': 28,
 'sausage': 29,
 'regular': 30,
 'big': 31,
 'meat': 32,
 'tomato': 33,
 'a': 34,
 'little': 35,
 'bit': 36,
 'pickle': 37,
 'stuffed': 38,
 'crust': 39,
 'lunch': 40,
 'pecorino': 41,
 'new': 42,
 'yorker': 43,
 'olive': 44,
 'oil': 45,
 'personal': 46,
 'bbq': 47,
 'basil': 48,
 'everything': 49,
 'mushroom': 50,
 'low': 51,
 'fat': 52,
 'pineaples': 53,
 'arugula': 54,
 'carrot': 55,
 'garlic': 56,
 'alfredo': 57,
 'napolitana': 58,
 'spinach': 59,
 'ham': 60,
 'spiced': 61,
 'bay': 62,
 'leaf': 63,
 'balsamic': 64,
 'glaze': 65,
 'yellow': 66,
 'salami': 67,
 'kal

In [17]:
vocab

{'<PAD>': 0,
 '<SOS>': 1,
 '<EOS>': 2,
 'party': 3,
 'sized': 4,
 'high': 5,
 'rise': 6,
 'dough': 7,
 'pie': 8,
 'with': 9,
 'american': 10,
 'cheese': 11,
 'extra': 12,
 'oregano': 13,
 'meatlover': 14,
 'chicken': 15,
 'medium': 16,
 'artichoke': 17,
 'large': 18,
 'green': 19,
 'pepper': 20,
 'pepperoni': 21,
 'roasted': 22,
 'mozzarella': 23,
 'ranch': 24,
 'sauce': 25,
 'banana': 26,
 'meatball': 27,
 'italian': 28,
 'sausage': 29,
 'regular': 30,
 'big': 31,
 'meat': 32,
 'tomato': 33,
 'a': 34,
 'little': 35,
 'bit': 36,
 'pickle': 37,
 'stuffed': 38,
 'crust': 39,
 'lunch': 40,
 'pecorino': 41,
 'new': 42,
 'yorker': 43,
 'olive': 44,
 'oil': 45,
 'personal': 46,
 'bbq': 47,
 'basil': 48,
 'everything': 49,
 'mushroom': 50,
 'low': 51,
 'fat': 52,
 'pineaples': 53,
 'arugula': 54,
 'carrot': 55,
 'garlic': 56,
 'alfredo': 57,
 'napolitana': 58,
 'spinach': 59,
 'ham': 60,
 'spiced': 61,
 'bay': 62,
 'leaf': 63,
 'balsamic': 64,
 'glaze': 65,
 'yellow': 66,
 'salami': 67,
 'kal

In [18]:
X_train_processed.shape, y_train_processed.shape

((476370, 270), (476370, 270))

In [19]:
X_train_processed[1]

array([14,  8,  9, 12, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0

In [20]:
y_train_processed[1]

array([235, 236, 238,  14,   8,   9, 240, 241,  12, 239,  15,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [21]:

ou= decode_sequence(y_train_processed[1],vocab)
ou

'ORDER PIZZAORDER STYLE meatlover pie with COMPLEX_TOPPING QUANTITY extra TOPPING chicken'

In [22]:
y_train[1]

'ORDER PIZZAORDER STYLE meatlover pie with COMPLEX_TOPPING QUANTITY extra TOPPING chicken'

In [23]:
ou == y_train[1]

True

### Ensure Decoding works right

In [24]:
total_sequences = 0
correct_sequences = 0

for src,tgt in zip(y_test_processed, y_test):
    #print(src)

    predicted_sequences = decode_sequence(src, vocab) 
    #print(predicted_sequences)
    # print(predicted_sequences)
    # print(target_sequences)
    # Calculate sequence accuracy
    if predicted_sequences == tgt:
        # print(pred)
        # print(tgt)
        correct_sequences += 1
    total_sequences += 1
print(f"Correct {correct_sequences}, Total {total_sequences}")
sequence_accuracy = correct_sequences / total_sequences if total_sequences > 0 else 0
sequence_accuracy * 100

Correct 53, Total 348


15.229885057471265

In [25]:
len(vocab)

247

In [26]:
input_dim = X_train_processed.shape[1]
input_dim

270

In [27]:
output_dim = y_train_processed.shape[1] 
output_dim

270

In [57]:
input_dim =  X_train_processed.shape[1]  # Vocabulary size
embedding_dim = 247  # Dimension of embedding vectors    vocabs  260,158       max seq lens  (99, 265)
hidden_dim = 256  # Hidden state size for LSTM          first was 256 for batch norm 150
output_dim = y_train_processed.shape[1]  # Number of output classes
num_layers = 2  # Number of BiLSTM layers
dropout = 0.2  # Dropout probability

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GRUModel(input_dim, embedding_dim, hidden_dim, output_dim, num_layers, dropout).to(device)


In [58]:
import torch
from torch.utils.data import Dataset, DataLoader

class SequenceDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = torch.tensor(inputs, dtype=torch.long) 
        self.targets = torch.tensor(targets, dtype=torch.long) 

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return {
            "src_input_ids": self.inputs[idx],
            "tgt_input_ids": self.targets[idx],
        }

train_dataset = SequenceDataset(X_train_processed, y_train_processed)
test_dataset = SequenceDataset(X_test_processed, y_test_processed)

batch_size = 64  # Adjust based GPU ;-;  memory
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [47]:
train_dataset.__getitem__(3)

{'src_input_ids': tensor([18,  8,  9, 19, 20, 21,  9, 12, 22, 20,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0

In [59]:
def evaluate_model_with_accuracy(model, dataloader, criterion, device):
    model.eval()
    epoch_loss = 0
    total_tokens = 0
    correct_tokens = 0

    with torch.no_grad():
        for batch in dataloader:
            src = batch["src_input_ids"].to(device)
            tgt = batch["tgt_input_ids"].to(device)

            output = model(src)
            output_dim = output.shape[-1]

            # Flatten outputs and targets
            output = output.view(-1, output_dim)  # Shape: (batch_size * seq_len, output_dim)
            tgt = tgt.view(-1)  # Shape: (batch_size * seq_len)

            # # Apply mask to remove padding tokens
            # mask = tgt != 0  # Mask to ignore padding indices
            # output = output[mask]  # Filter model outputs
            # tgt = tgt[mask]  # Filter targets

            # Compute loss
            loss = criterion(output, tgt)
            epoch_loss += loss.item()

            # Calculate accuracy
            predictions = output.argmax(dim=1)  # Get the index of the max log-probability
            correct_tokens += (predictions == tgt).sum().item()
            total_tokens += tgt.size(0)

    accuracy = correct_tokens / total_tokens if total_tokens > 0 else 0
    return epoch_loss / len(dataloader), accuracy


In [60]:
import torch.optim as optim
from tqdm import tqdm

criterion = nn.CrossEntropyLoss()  # Use for multi-class classification ignore_index=0 for padding
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

for epoch in range(20):  # Number of epochs
    model.train()
    epoch_loss = 0
    total_batches = len(train_dataloader)
    progress_bar = tqdm(train_dataloader, desc="Training Progress", unit="batch", leave=True)

    for batch_idx, batch in enumerate(progress_bar):  # Assuming a DataLoader is used
        src = batch["src_input_ids"].to(device)  # Input tokens
        tgt = batch["tgt_input_ids"].to(device)  # Target tokens

        optimizer.zero_grad()
        output = model(src)  # Forward pass
        output_dim = output.shape[-1]

        # Flatten outputs and targets for loss computation
        output = output.view(-1, output_dim)  # Shape: (batch_size * seq_len, output_dim)
        tgt = tgt.view(-1)  # Shape: (batch_size * seq_len)

        # Apply mask to remove padding tokens
        # mask = tgt != 0  # Mask to ignore padding indices
        # output = output[mask]  # Filter model outputs
        # tgt = tgt[mask]  # Filter targets
        # print(output.shape)
        # print(tgt.shape)

        # Compute loss
        loss = criterion(output, tgt)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        avg_loss = epoch_loss / (batch_idx + 1)
        progress_bar.set_description(f"Training Progress: Batch {batch_idx + 1}/{total_batches}, Avg Loss: {avg_loss:.8f}")

    val_loss, accuracy = evaluate_model_with_accuracy(model, test_dataloader, criterion, device)
    print(f"Epoch {epoch + 1}, Loss: {epoch_loss / len(train_dataloader):.8f}, Val Loss: {val_loss:.8f}, Accuracy: {accuracy * 100:.4f}%")


Training Progress: Batch 7444/7444, Avg Loss: 0.0245: 100%|██████████| 7444/7444 [05:50<00:00, 21.21batch/s]


Epoch 1, Loss: 0.0245, Val Loss: 0.2983, Accuracy: 96.82%


Training Progress: Batch 7444/7444, Avg Loss: 0.0019: 100%|██████████| 7444/7444 [05:54<00:00, 20.97batch/s]


Epoch 2, Loss: 0.0019, Val Loss: 0.2658, Accuracy: 96.92%


Training Progress: Batch 7444/7444, Avg Loss: 0.0014: 100%|██████████| 7444/7444 [06:04<00:00, 20.44batch/s]


...

Epoch 7, Loss: 0.0009, Val Loss: 0.2425, Accuracy: 97.15%


Training Progress: Batch 7444/7444, Avg Loss: 0.0009: 100%|██████████| 7444/7444 [05:57<00:00, 20.81batch/s]


Epoch 8, Loss: 0.0009, Val Loss: 0.2601, Accuracy: 97.04%


Training Progress: Batch 140/7444, Avg Loss: 0.0009:   2%|▏         | 140/7444 [00:06<05:57, 20.41batch/s]


KeyboardInterrupt: 

### Saving the model

In [61]:
def save_model(model, path):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

def load_model(model, path):
    model.load_state_dict(torch.load(path))
    print(f"Model loaded from {path}")
    return model

In [62]:
save_model(model, "../weights/GRU.pt")

Model saved to ../weights/GRU.pt


In [ ]:
model = BiLSTMModel(input_dim, embedding_dim, hidden_dim, output_dim, num_layers, dropout).to(device)
model = load_model(model,"../weights/GRU.pt")

Model loaded from ../weights/Bilstm_2layers_withpadding.pt


### Testing Real Output sequence

In [63]:
test_dataset.__getitem__(1)["src_input_ids"]

tensor([105,  16,  81,   9,  33,  60,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  

In [64]:

with torch.no_grad():

    output = model( test_dataset.__getitem__(1)["src_input_ids"].to(device))
    predictions = output.argmax(dim=1)
predictions

RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 2 is not equal to len(dims) = 3

In [65]:
tgt= test_dataset.__getitem__(1)["tgt_input_ids"].to(device)
tgt

tensor([235, 236, 242, 105, 237,  16,  81,   9, 239,  33, 239,  60,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  

In [39]:

sequence = predictions.cpu().tolist()
pred_sequence= decode_sequence(sequence,vocab)
pred_sequence

NameError: name 'predictions' is not defined

In [40]:
sequence = tgt.cpu().tolist()
tgt_sequence= decode_sequence(sequence,vocab)
tgt_sequence

'ORDER PIZZAORDER NUMBER five SIZE medium pizza with TOPPING tomato TOPPING ham'

In [41]:
pred_sequence == tgt_sequence

NameError: name 'pred_sequence' is not defined

In [66]:
def evaluate_model_with_sequence_accuracy(model, dataloader, device):
    model.eval()
    total_sequences = 0
    correct_sequences = 0

    with torch.no_grad():
        for batch in dataloader:
            src = batch["src_input_ids"].to(device)
            tgt = batch["tgt_input_ids"].to(device)

            output = model(src)
            output = output.argmax(dim=-1)

            # Decode sequences for comparison

            predicted_sequences = [decode_sequence(seq.cpu().tolist(), vocab) for seq in output]
            target_sequences = [decode_sequence(seq.cpu().tolist(), vocab) for seq in tgt]
            # print(predicted_sequences)
            # print(target_sequences)
            # Calculate sequence accuracy
            for pred, tgt in zip(predicted_sequences, target_sequences):
                if pred == tgt:
                    # print(pred)
                    # print(tgt)
                    correct_sequences += 1
                else:
                    print(pred)
                    print(tgt)
                total_sequences += 1

    print(f"Correct {correct_sequences}, Total {total_sequences}")
    sequence_accuracy = correct_sequences / total_sequences if total_sequences > 0 else 0
    return sequence_accuracy * 100

In [67]:
evaluate_model_with_sequence_accuracy(model, test_dataloader, device)

ORDER DRINKORDER NUMBER two SIZE medium pizza with TOPPING sausage TOPPING black olive PIZZAORDER NUMBER NUMBER two SIZE medium pizza with TOPPING cheese cheese DRINKORDER powder powder sausage
ORDER PIZZAORDER NUMBER two SIZE medium pizza with TOPPING sausage TOPPING black olive PIZZAORDER NUMBER two SIZE medium pizza with TOPPING pepperoni COMPLEX_TOPPING QUANTITY extra TOPPING cheese PIZZAORDER NUMBER three SIZE large pizza with TOPPING pepperoni TOPPING sausage
ORDER DRINKORDER NUMBER five SIZE medium pizza with TOPPING tomato TOPPING ham
ORDER PIZZAORDER NUMBER five SIZE medium pizza with TOPPING tomato TOPPING ham
ORDER PIZZAORDER PIZZAORDER NUMBER one SIZE large large pizza with TOPPING QUANTITY extra TOPPING banana pepper
ORDER need PIZZAORDER NUMBER one SIZE large STYLE vegetarian pizza with COMPLEX_TOPPING QUANTITY extra TOPPING banana pepper
ORDER like PIZZAORDER NUMBER a SIZE large onion onion NUMBER NUMBER DRINKTYPE ice tea
ORDER like PIZZAORDER NUMBER a SIZE large TOPPING

6.0344827586206895